In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import heapq
import pickle


# usando o arquivo '1661-0.txt' que é muito utilizado para desenvolvimento de uma LSTM
path = '1661-0.txt'
text = open(path).read().lower()
print('quantidade de palavras: ', len(text))

quantidade de palavras:  581533


In [ ]:
# Tokenização para apenas os caracteres alfanuméricos serem considerados como parte dos tokens 
# E cada token ser uma palavra do texto original guardada em word
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

# Criação do dicionário de palavras únicas
# O NumPy é usado para realizar essa operação de forma eficiente

unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

# Essa variável representa o número de palavras consecutivas que serão usadas como entrada
WORD_LENGTH = 5

# Essas variáveis são usadas em um loop posterior para criar as sequências de palavras anteriores e suas respectivas palavras seguintes.
prev_words = []
next_words = []

# Geração dos pares de palavras de entrada e saída

for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])

print(prev_words[0], prev_words[1], prev_words[2])
print(next_words[0], next_words[1])


['the', 'project', 'gutenberg', 'ebook', 'of'] ['project', 'gutenberg', 'ebook', 'of', 'the'] ['gutenberg', 'ebook', 'of', 'the', 'adventures']
the adventures


In [ ]:
# criamos matrizes vazias X e Y
# onde X será usada para armazenar a codificação one-hot das sequências de palavras anteriores 
# Y será usada para armazenar a codificação one-hot das palavras seguintes correspondentes.
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

# preencher as matrizes X e Y 
# com os valores da codificação one-hot das sequências de palavras anteriores e suas respectivas palavras seguintes
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1


# Definição e compilação do modelo LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

# definir o otimizador RMSprop como o otimizador para o modelo.
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X, Y, validation_split=0.05, batch_size=256, epochs=5, shuffle=True)

# Função para preparar a entrada do modelo
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        x[0, t, unique_word_index[word]] = 1
    return x

prepare_input("It is not a lack".lower())

# Função para amostragem de palavras com base nas probabilidades preditas
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

# Função para prever as próximas palavras
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

# Teste de predição de palavras
q = "Your life will never be the same again"
print("correct sentence: ", q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ", seq)
print("next possible words: ", predict_completions(seq, 5))


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
406/406 [==============================] - 271s 660ms/step - loss: 6.0702 - accuracy: 0.0990 - val_loss: 6.9160 - val_accuracy: 0.0951
Epoch 2/5
406/406 [==============================] - 269s 662ms/step - loss: 5.4425 - accuracy: 0.1471 - val_loss: 7.0179 - val_accuracy: 0.1077
Epoch 3/5
406/406 [==============================] - 269s 663ms/step - loss: 5.2222 - accuracy: 0.1794 - val_loss: 7.5196 - val_accuracy: 0.1070
Epoch 4/5
406/406 [==============================] - 270s 664ms/step - loss: 5.0457 - accuracy: 0.2171 - val_loss: 8.1376 - val_accuracy: 0.1009
Epoch 5/5
406/406 [==============================] - 263s 649ms/step - loss: 4.8041 - accuracy: 0.2615 - val_loss: 8.0259 - val_accuracy: 0.0987
correct sentence:  Your life will never be the same again
Sequence:  your life will never be
next possible words:  ['a', 'an', 'the', 'your', 'to']
